In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.functions import explode, split

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 17:13:26 INFO SparkEnv: Registering MapOutputTracker
24/01/09 17:13:26 INFO SparkEnv: Registering BlockManagerMaster
24/01/09 17:13:26 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/01/09 17:13:26 INFO SparkEnv: Registering OutputCommitCoordinator


In [4]:
import pandas as pd

initial_approvals = pd.read_csv("../../data/2023/2023_approvals_in_v0.csv")
initial_approvals = spark.createDataFrame(initial_approvals)

initial_approvals = spark.read.option("header", "true").csv("../../data/2023/2023_approvals_in_v0.csv").persist()

initial_approvals.show()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 974: invalid continuation byte

In [26]:
# originalDrugName explode
initial_approvals = initial_approvals.withColumn("originalDrugNameArray", split(initial_approvals.originalDrugName, ","))
exploded_approvals = initial_approvals.withColumn("drugName", explode(initial_approvals.originalDrugNameArray))

# exploded_approvals.show()


### Finding drugId by drugName

In [28]:
molecule_path = "gs://open-targets-data-releases/23.12/output/etl/json/molecule"
molecule = spark.read.json(molecule_path)
molecule.persist()
molecule.show()

24/01/09 16:41:17 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://open-targets-data-releases/23.12/output/etl/json/molecule.
java.io.IOException: Error accessing gs://open-targets-data-releases/23.12/output/etl/json/molecule
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2235)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2122)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfoInternal(GoogleCloudStorageFileSystem.java:1104)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfo(GoogleCloudStorageFileSystem.java:1078)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.lambda$getFileStatus$9(GoogleHadoopFileSystemBase.j

Py4JJavaError: An error occurred while calling o98.json.
: java.io.IOException: Error accessing gs://open-targets-data-releases/23.12/output/etl/json/molecule
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2235)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2122)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfoInternal(GoogleCloudStorageFileSystem.java:1104)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfo(GoogleCloudStorageFileSystem.java:1078)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.lambda$getFileStatus$9(GoogleHadoopFileSystemBase.java:1040)
	at com.google.cloud.hadoop.fs.gcs.GhfsStorageStatistics.trackDuration(GhfsStorageStatistics.java:82)
	at com.google.clo